# Workshop Part 2: Solving Frequency Assignment with QUBO

Now it's your turn to apply what you've learned. We will tackle a classic optimization problem with a direct application in telecommunications: **Frequency Assignment** for mobile network cells.

### The Problem
Mobile network cells that are close to each other can interfere if they use the same frequency. To prevent this, we must assign frequencies to cells such that no two adjacent cells share the same frequency. This is a classic **Graph Coloring** problem in disguise.

- **Cells** are the **nodes** of our graph.
- **Frequencies** are the **colors** we can use.
- An **edge** exists between two nodes if their cells are close enough to interfere.

Our goal is to find a valid frequency assignment for a small honeycomb network of cells.

## Step 1: Setup and Dependencies

First, we import the necessary libraries. We'll use `matplotlib.patches` to draw our honeycomb cells.

In [ ]:
# Core scientific computing and plotting libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import RegularPolygon
import networkx as nx # Still useful for managing graph data

# pyqubo for building the QUBO model
from pyqubo import Array, Constraint, Placeholder

# neal for the simulated annealing solver
import neal

print("Libraries imported successfully!")

## Step 2: Problem Definition

Let's define our network as a 7-cell honeycomb grid. We have 3 available frequencies (e.g., 'Channel 1', 'Channel 2', 'Channel 3').

In [ ]:
# Define the cells (nodes) and available frequencies (colors)
cells = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
frequencies = ['Red', 'Green', 'Blue'] # Using colors for visualization

n_cells = len(cells)
n_frequencies = len(frequencies)

# Define the honeycomb graph structure
# 'A' is the central cell
edges = [
    ('A', 'B'), ('A', 'C'), ('A', 'D'), ('A', 'E'), ('A', 'F'), ('A', 'G'),
    ('B', 'C'), ('C', 'D'), ('D', 'E'), ('E', 'F'), ('F', 'G'), ('G', 'B')
]

# Create a graph object to hold the data
G = nx.Graph()
G.add_nodes_from(cells)
G.add_edges_from(edges)

print(f"Problem defined for {n_cells} cells and {n_frequencies} frequencies.")

### Visualizing the Network

Let's visualize the cell network. We will define the positions of the nodes to create a perfect honeycomb and then draw the hexagons using `matplotlib`.

In [ ]:
def get_honeycomb_pos(graph):
    """Calculates node positions for a 7-cell honeycomb layout."""
    pos = {'A': (0, 0)} # Central node
    outer_nodes = [node for node in graph.nodes() if node != 'A']
    for i, node_name in enumerate(outer_nodes):
        angle = np.pi / 3 * i
        pos[node_name] = (np.cos(angle), np.sin(angle))
    return pos

def plot_network(graph, solution=None):
    """Helper function to draw a honeycomb and the coloring solution."""
    fig, ax = plt.subplots(1, figsize=(8, 8))
    ax.set_aspect('equal')
    pos = get_honeycomb_pos(graph)

    for node_name, center in pos.items():
        color = solution.get(node_name, 'lightblue') if solution else 'lightblue'
        
        # Draw the hexagon
        hexagon = RegularPolygon(
            center, 
            numVertices=6, 
            radius=1 / np.sqrt(3) * 1.01, # Calculate radius for touching hexagons
            orientation=np.pi / 6, 
            facecolor=color, 
            edgecolor='black', 
            linewidth=2.5
        )
        ax.add_patch(hexagon)
        # Add the cell label
        ax.text(center[0], center[1], node_name, ha='center', va='center', fontsize=24, color='white', weight='bold')

    ax.set_xlim(-1.5, 1.5)
    ax.set_ylim(-1.5, 1.5)
    plt.axis('off')
    plt.title("Mobile Cell Network (Honeycomb)" if not solution else "Frequency Assignment Solution", fontsize=16)
    plt.show()

# Plot the initial problem
plot_network(G)

## Step 3: QUBO Formulation

Now we translate the frequency assignment problem into a QUBO model. This involves defining our binary variables, the objective function (penalizing bad assignments), and the constraints.

### 3.1 Decision Variables

We need a binary variable that decides which cell is assigned which frequency  

 **Your task is to create this array of variables.**

In [ ]:
# --- Your code goes here --- #
# Create a 2D array of binary variables called 'x'.
# It should have a shape of (n_cells, n_frequencies).
x = # ...

### 3.2 The Objective Function

Our main goal is to avoid interference. This means that for any two cells `u` and `v` that are connected by an edge, they cannot have the same frequency.

**Your task is to build this objective function.**

In [ ]:
# --- Your code goes here --- #
# Create a variable 'objective_func'.
# Loop through each edge (u_str, v_str) in G.edges().
# Inside the loop, get the integer indices 'u' and 'v' for the cells.
# Then, loop through each frequency 'c' from 0 to n_frequencies-1.
# Add the product of the variables x[u, c] * x[v, c] to 'objective_func'.
objective_func = # ...

### 3.3 The Constraint

We have one main rule that must be strictly followed:

**Every cell must be assigned exactly one frequency.**


**Your task is to build this constraint function.**

In [ ]:
# Use a Placeholder for the penalty weight
P = Placeholder("penalty_weight")

# --- Your code goes here --- #
# Create a variable 'constraint_func'.
# Loop through each cell 'v' from 0 to n_cells-1.
# For each cell, create a Constraint where the sum of its frequency variables (x[v, c]) equals 1.
# Add this constraint to 'constraint_func'.
constraint_func = 0
for v in range(n_cells):
    constraint_func += Constraint((sum(x[v, c] for c in range(n_frequencies)) - 1)**2, label=f"cell_visited_{v}")

# Construct the final Hamiltonian 'H' by adding the objective_func
# and the constraint_func (multiplied by the penalty 'P').
H = objective_func + P * constraint_func

### 3.4 The Final Hamiltonian

Now we combine the objective and the constraint into our final model. The solver's goal will be to find an assignment that minimizes this total value. A perfect solution will have an energy of 0.

**Your task is to combine the objective and constraint.**

In [ ]:
# --- Your code goes here --- #
# Construct the final Hamiltonian 'H' by adding the objective_func
# and the constraint_func (multiplied by the penalty 'P').
H = # ...

## Step 4: Compile and Solve the QUBO

With our Hamiltonian defined, we compile it into a QUBO matrix and send it to the simulated annealer to solve.

**Your task is to complete the solving process.**

In [ ]:
# --- Your code goes here --- #

# 1. Compile the model 'H'
model = # ...

# 2. Define the penalty value in a feed_dict.
# A value of 2.0 is a good starting point.
feed_dict = # ...

# 3. Convert the compiled model to a QUBO dictionary
qubo, offset = # ...

# 4. Create a sampler and solve the QUBO
sampler = # ...
sampleset = # ...

# 5. Decode the best solution found
decoded_sample = # ...

print(f"Solver finished. Lowest energy found: {decoded_sample.energy}")

## Step 5: Interpret and Visualize the Solution

Finally, we parse the solver's output to determine the frequency for each cell and check if our solution is valid.

**Your task is to write the code to interpret the solution.**

In [ ]:
# --- Your code goes here --- #

# Check for broken constraints in the decoded_sample.
# If there are no broken constraints:
#  1. Create an empty dictionary called 'solution'.
#  2. Loop through cells and frequencies to find which x[v, c] is 1.
#  3. Populate the 'solution' dictionary with the cell name and assigned frequency.
#  4. Print the assignments.
#  5. Call plot_network(G, solution) to see your result!
#
# If there are broken constraints, print an error message.